In [46]:
import json
import ast
from typing import List
import requests
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
import pandas as pd
from PIL import Image

In [47]:
cols = ["item_id", "title", "genres_data", "countries_data", "release_year", "rating", "poster", 'user_id']
# paths
api_response_path = "../data/external/found_films/{}_info.json"
preds_path = "../reports/predictions.csv"
films_path = "../data/interim/5k_selected_items.csv"
history_path = "../data/interim/selected_interactions.csv"

def process_image_url(url: str):
	template = "<img src='{url}' width='120' height='180'>"
	if url:
		return template.format(url=url)
	else:
		return "<img src='None' width='120' height='180'>"
	
def load_api_info(ids: List[int]):
	arr = []
	for cur_id in ids:
		try:
			with open (api_response_path.format(cur_id), "r") as f:
				data = pd.DataFrame.from_dict({f"{cur_id}": json.load(f)}, orient="index")
				arr.append(data)
		except Exception:
			pass
	return arr

In [48]:
# load data
films_df = pd.read_csv(films_path)
predictions = pd.read_csv(preds_path)
history_df = pd.read_csv(history_path)
with open("users_to_show.json", 'r') as f:
	users_to_show = json.load(f)['users']

# recs
users_to_show = list(map(int, users_to_show))
predictions = predictions[predictions['user_id'].isin(users_to_show)]
predictions['recs'] = predictions['recs'].apply(lambda x: ast.literal_eval(x))
predictions = predictions.explode('recs')
recommendations = pd.merge(predictions, films_df, left_on='recs', right_on='item_id')
recs_ids = recommendations['item_id'].unique().tolist()
recommendations['item_id'] = recommendations['item_id'].astype('int')

# story data
history_df = history_df[history_df['user_id'].isin(users_to_show)].loc[:, ['user_id', 'item_id']]
history_df = pd.merge(history_df, films_df, on='item_id')
history_ids = history_df['item_id'].unique().tolist()


In [49]:
# recs info
recs_info_from_api = load_api_info(recs_ids)
recs_info_from_api = pd.concat(recs_info_from_api).reset_index()
recs_info_from_api['index'] = recs_info_from_api['index'].astype(int)
recommendations = pd.merge(recommendations, recs_info_from_api, how="left", left_on="item_id", right_on="index", suffixes=("_data", "_api")).loc[:, cols]
recommendations['rating'] = recommendations['rating'].apply(lambda x: x.get('kp') if isinstance(x, dict) else x)
recommendations['poster'] = recommendations['poster'].apply(lambda x: x.get('url') if isinstance(x, dict) else x)
recommendations['poster'] = recommendations['poster'].apply(process_image_url)

# history info
history_info_from_api = load_api_info(history_ids)
history_info_from_api = pd.concat(history_info_from_api).reset_index()
history_info_from_api['index'] = history_info_from_api['index'].astype(int)
history_df = pd.merge(history_df, history_info_from_api, how="left", left_on="item_id", right_on="index", suffixes=("_data", "_api")).loc[:, cols]
history_df['rating'] = history_df['rating'].apply(lambda x: x.get('kp') if isinstance(x, dict) else x)
history_df['poster'] = history_df['poster'].apply(lambda x: x.get('url') if isinstance(x, dict) else x)
history_df['poster'] = history_df['poster'].apply(process_image_url)

/tmp/ipykernel_70409/2282514057.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  recs_info_from_api = pd.concat(recs_info_from_api).reset_index()
/tmp/ipykernel_70409/2282514057.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  history_info_from_api = pd.concat(history_info_from_api).reset_index()


In [56]:
recommendations_output = widgets.Output()
history_output = widgets.Output()

def display_recommendations(user_id: int):
    with recommendations_output:
        recommendations_output.clear_output()
        user_recs = recommendations[recommendations['user_id']==int(user_id)]
        html_recs = user_recs.drop(columns=['item_id', 'user_id']).loc[:, ['poster', 'title', 'genres_data', 'countries_data', 'release_year', 'rating']].to_html(escape=False, index=False, border=1)
        display(HTML(html_recs))

def display_history(user_id: int):
    with history_output:
        history_output.clear_output()
        user_history = history_df[history_df['user_id']==int(user_id)]
        html_history = user_history.drop(columns=['item_id', 'user_id']).loc[:, ['poster', 'title', 'genres_data', 'countries_data', 'release_year', 'rating']].to_html(escape=False, index=False, border=1)
        display(HTML(html_history))

In [58]:
buttons = []
for user in users_to_show:
    button = widgets.Button(description=str(user), button_style='warning')
    buttons.append(button)

for button in buttons:
    button.on_click(lambda b: display_recommendations(int(b.description)))
    button.on_click(lambda b: display_history(int(b.description)))
    
buttons_box = widgets.HBox(buttons)


In [62]:
display(widgets.VBox([
	buttons_box,
    widgets.HTML("<h3>История просмотров пользователя:</h3>"),
    history_output,
    widgets.HTML("<hr style='height: 2px; color: black;'>"),
    widgets.HTML("<h3>Рекомендации для пользователя:</h3>"),
    recommendations_output
]))

style = """
    <style>
    .my-table {
        width: 100%;
        table-layout: fixed;
    }
    .my-table th, .my-table td {
        padding: 10px;
        text-align: center;
    }
    </style>
"""

display(widgets.HTML(style))

HTML(value='\n    <style>\n    .my-table {\n        width: 100%;\n        table-layout: fixed;\n    }\n    .my…